In [19]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/401k.csv')


In [20]:
df.dtypes

nifa       int64
net_tfa    int64
tw         int64
age        int64
inc        int64
fsize      int64
educ       int64
db         int64
marr       int64
twoearn    int64
e401       int64
p401       int64
pira       int64
hown       int64
dtype: object

In [21]:
outcome = 'e401'
treatment = 'net_tfa'
rest = list(df.drop([outcome, treatment], axis = 1).columns)
#df[treatment] = 100000* df[treatment]

#rest = ['exper','age', 'kidslt6', 'kidsge6']
df = df[[outcome] + [treatment] + rest]

In [22]:
#df = df.dropna()
#df = df.fillna(0)
#df = df[df.avg <= 15]
y = df[outcome].astype('float')
d = df[treatment].astype('float')
x = df[rest].astype('float')
print(df.shape)
x.head()

(9915, 14)


,nifa,tw,age,inc,fsize,educ,db,marr,twoearn,p401,pira,hown
0,0.0,4500.0,47.0,6765.0,2.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0
1,6215.0,22390.0,36.0,28452.0,1.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,-2000.0,37.0,3300.0,6.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0
3,15000.0,155000.0,58.0,52590.0,2.0,16.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.0,58000.0,32.0,21804.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                   e401   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     224.4
Date:                Mon, 05 Dec 2022   Prob (F-statistic):           3.51e-50
Time:                        03:18:04   Log-Likelihood:                -6745.7
No. Observations:                9915   AIC:                         1.350e+04
Df Residuals:                    9913   BIC:                         1.351e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.132e-06   7.55e-08     14.980      0.0

In [24]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                   e401   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     1256.
Date:                Mon, 05 Dec 2022   Prob (F-statistic):               0.00
Time:                        03:18:05   Log-Likelihood:                -2026.2
No. Observations:                9915   AIC:                             4080.
Df Residuals:                    9901   BIC:                             4181.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -5.202e-07   1.67e-07     -3.112      0.0

In [25]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 200, max_features = 'sqrt', max_depth= 8)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: e401
Treatment variable(s): ['net_tfa']
Covariates: ['nifa', 'tw', 'age', 'inc', 'fsize', 'educ', 'db', 'marr', 'twoearn', 'p401', 'pira', 'hown']
Instrument variable(s): None
No. Observations: 9915

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9915 entries, 0 to 9914
Columns: 14 entries, e401 to hown
dtypes: int64(14)
memory usage: 1.1 MB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: e401
Treatment variable(s): ['net_tfa']
Covariates: ['nifa', 'tw', 'age', 'inc', 'fsize', 'educ', 'db', 'marr', 'twoearn', 'p401', 'pira', 'hown']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Mac

In [26]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=list(rest))
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: e401
Treatment variable(s): ['net_tfa']
Covariates: ['nifa', 'tw', 'age', 'inc', 'fsize', 'educ', 'db', 'marr', 'twoearn', 'p401', 'pira', 'hown']
Instrument variable(s): None
No. Observations: 9915

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9915 entries, 0 to 9914
Columns: 14 entries, e401 to hown
dtypes: int64(14)
memory usage: 1.1 MB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: e401
Treatment variable(s): ['net_tfa']
Covariates: ['nifa', 'tw', 'age', 'inc', 'fsize', 'educ', 'db', 'marr', 'twoearn', 'p401', 'pira', 'hown']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Mac

In [27]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,0.0,0.0,9.53,0.0,0.0,0.0
